In [ ]:
#%pip install gym==0.22.0

In [ ]:
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [ ]:
env = gym.make('Pendulum-v1')
#Pendulum-v1 是一个经典的控制任务，其中智能体（agent）必须学会如何控制一个倒立摆，使其保持在垂直位置。

EP_MAX = 1000
#EP_MAX: 这个变量代表训练过程中要运行的最大周期数（episodes）。

HORIZON = 128
#HORIZON: 这个变量定义了每个周期中的最大步数（time steps）。

LR_v = 2e-5
#LR_v: 这是评价者（Critic）网络的学习率。

LR_pi = 2e-5
#LR_pi: 这是行动者（Actor）网络的学习率。

K_epoch = 8
#K_epoch: 这个变量表示每个周期中进行的优化迭代次数。

GAMMA = 0.99
#GAMMA: 这是折扣因子（discount factor），用于计算未来回报的当前价值。

LAMBDA = 0.95
#LAMBDA: 这是用于计算广义优势估计（Generalized Advantage Estimation, GAE）的参数。

CLIP = 0.2
#CLIP: 这是 PPO（Proximal Policy Optimization）算法中的裁剪参数。

In [ ]:
class Pi_net(nn.Module):
#定义了一个新的类 Pi_net，它继承自 torch.nn.Module，这是所有神经网络模块的基类。

    def __init__(self):
        #Pi_net 类的构造函数，用于初始化网络。

        super(Pi_net, self).__init__()
        #调用父类（即 nn.Module）的构造函数，这是在Python中继承时的标准做法。

        self.net = nn.Sequential(
        #定义了一个顺序容器 nn.Sequential，它将按照顺序包含多个神经网络层。
        
            nn.Linear(3, 64),
            #定义了一个全连接层，它将输入维度从3增加到64。这通常是环境状态的大小。

            nn.ReLU(),
            #定义了一个ReLU激活函数，它将应用于前一个全连接层的输出。

            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
        )
        #这里包含了三个全连接层（nn.Linear）和两个ReLU激活函数（nn.ReLU）。

        self.mu = nn.Linear(256, 1)
        #定义了另一个全连接层，它将最后一个隐藏层的输出（维度256）映射到一个单一的输出，这个输出代表策略分布的均值（mu）。

        self.sigma = nn.Linear(256, 1)
        #定义了另一个全连接层，用于估计策略分布的标准差（sigma）。

        self.optim = torch.optim.Adam(self.parameters(), lr=LR_pi)
        #创建了一个Adam优化器，用于后续的网络参数更新。
        #这里的 self.parameters() 是 nn.Module 类的方法，它将返回所有神经网络层的参数。
        #lr=LR_pi 设置了学习率为 LR_pi。
 
    def forward(self, x):
        #定义了网络的前向传播函数，它接受输入 x。

        x = self.net(x)
        #将输入 x 通过定义好的网络层进行前向传播。
        
        mu = torch.tanh(self.mu(x)) * 2
        #计算动作的均值。
        #首先通过全连接层 self.mu 计算原始输出
        #然后应用 torch.tanh 函数将输出限制在[-1, 1]区间内，最后乘以2以扩展到环境可能允许的动作范围。

        sigma = F.softplus(self.sigma(x)) + 0.001
        #计算动作的标准差。
        #首先通过全连接层 self.sigma 计算原始输出，然后应用 softplus 函数（F.softplus）
        #这是一个平滑的ReLU函数，用于确保标准差是正数。
        #最后加上0.001以避免标准差为零，保证策略分布的方差。

        return mu, sigma

神经网络这一块有点没搞懂

在调用 __init__ 方法时，Python 会自动将新创建的实例作为第一个参数传递给这个方法。按照惯例，这个参数被命名为 self。

self 是一个特殊的变量，它在类的方法内部用来引用当前对象本身。

Pi_net 类代码中，self 用于以下方面：

self.net：定义了一个神经网络层的序列，属于当前 Pi_net 实例。
self.mu 和 self.sigma：定义了两个全连接层，它们也是当前实例的一部分，用于计算策略的均值和标准差。
self.optim：创建了一个优化器，它是实例的一部分，用于在训练过程中更新网络的参数。
self.parameters()：在优化器中使用，返回当前实例的所有参数，这些参数将被优化器用于梯度下降。
self.forward(x)：定义了前向传播方法，self 在这里用于访问网络层和激活函数，以计算输出。



在PyTorch框架中，nn.Module 是所有神经网络模块的基类，它提供了一些基础功能，例如：

管理模块的参数（通过 self.parameters()）。
跟踪子模块（通过 self.children()）。
定义前向传播（通过 self.forward()）。
支持钩子函数（如 forward_pre_hooks 和 forward_hook）。

nn.Linear 是 PyTorch 中的一个模块（nn.Module 的子类），它实现了一个全连接的线性层，也就是一个神经网络中最基本的计算单元。这个层接收一个输入张量，并将其通过一个线性函数，
其中 x 是输入，W 是权重矩阵，b 是偏置向量，y 是输出。

nn.Linear 模块会自动创建两个参数：

weight：一个形状为 [out_features, in_features] 的矩阵，其中 in_features 是输入特征的数量（3），out_features 是输出特征的数量（64）。
bias：一个形状为 [out_features] 的向量，但在某些情况下可以设置为不包含偏置（通过将 bias=False 传递给 nn.Linear 构造函数）。

`nn.Linear` 通过权重矩阵（`weight`）和偏置向量（`bias`）来从输入特征 "变出" 更多的特征。这个过程可以用数学公式来表示：

output = input * weight ^ T + bias

这里的操作和概念包括：

1. 权重矩阵 weight：这是一个二维矩阵，其行数等于输出特征数（在这个例子中是64），列数等于输入特征数（在这个例子中是3）。权重矩阵用于将输入特征转换为输出特征。

2. 偏置向量 bias：这是一个一维向量，其长度等于输出特征的数量。偏置向量为每个输出特征提供了一个可学习的偏移量。

3. 矩阵乘法：输入数据首先被视为一个矩阵（或向量的集合），然后与权重矩阵进行矩阵乘法。这个过程将输入特征的每个维度映射到输出特征空间。

4. 转置操作 weight：权重矩阵需要被转置，以便能够与输入矩阵相乘。如果输入是一个批量大小为 \( m \) 的三维张量（形状为 `(batch_size, in_features)`），权重矩阵将被转置以匹配这个形状。

5. 广播机制：偏置向量通过广播机制添加到矩阵乘法的结果上，确保每个输出特征都有一个偏置项。

6. 非线性激活函数：在许多情况下，线性层的输出会通过一个非线性激活函数（如ReLU），这为网络增加了非线性能力，允许它学习和模拟更复杂的函数映射。

7. 参数学习：在训练过程中，权重矩阵和偏置向量是可学习的参数，它们通过反向传播算法根据损失函数的梯度进行更新。

通过这种方式，`nn.Linear` 能够将输入特征转换为更高维度的输出特征空间，这为构建深度神经网络和学习复杂的模式提供了基础。这种从较少特征到较多特征的转换是深度学习模型能够表示和学习数据中复杂结构的关键部分。


这个 Pi_net 类定义了一个神经网络模型，用于在强化学习中作为策略网络。

这个网络模型是用于输出给定状态的动作分布参数的，

具体来说，它输出了两个参数：均值（mu）和标准差（sigma），这两个参数定义了一个高斯分布（正态分布），在连续动作空间中用来采样动作。

问题：

为什么在连续动作空间中，会以来高斯分布来采样动作？
可微性：高斯分布的参数（均值和标准差）是可微的，这对于使用梯度下降算法进行优化至关重要。
可解释性：高斯分布提供了对动作分布的直观理解，均值可以解释为在给定状态下最可能采取的动作，而标准差可以解释为动作的不确定性或变化范围。
探索与利用：在强化学习中，探索（exploration）和利用（exploitation）是两个关键概念。高斯分布可以通过调整标准差来平衡这两者，较大的标准差可以增加探索性，而较小的标准差可以增加利用性。
奖励函数的兼容性：在某些情况下，奖励函数可能对连续动作空间中的小变化非常敏感。使用高斯分布可以通过调整标准差来控制这种敏感性。

什么是神经网络模型？神经网络模型是怎么在PPO中使用的？为什么策略函数要使用神经网络模型？
1.
神经网络模型的主要特点：
层次结构：神经网络由多个层次组成，通常包括输入层、一个或多个隐藏层和输出层。
神经元：每个层级由多个神经元组成，每个神经元可以接收来自前一层神经元的输入，进行加权求和，并通过一个激活函数生成输出。
权重和偏置：每个连接都有一个权重，神经元还有一个偏置项。权重和偏置是神经网络的参数，它们在训练过程中通过学习数据进行调整。
激活函数：激活函数决定了神经元是否以及如何对输入信号做出反应。常见的激活函数包括ReLU、Sigmoid、Tanh等。
前向传播：数据在神经网络中从输入层经过每一层的神经元，通过加权求和和激活函数，最终在输出层生成预测结果。
损失函数：损失函数衡量模型预测与实际结果之间的差异，用于指导模型的训练。
反向传播：在训练过程中，通过反向传播算法计算损失函数关于网络参数的梯度，这些梯度用于更新网络的权重和偏置。
优化算法：优化算法（如SGD、Adam、RMSprop等）用于根据梯度更新网络参数，以最小化损失函数。



In [ ]:
class V_net(nn.Module):
    def __init__(self):
        super(V_net, self).__init__()
        #固定操作
        
        self.net = nn.Sequential(
            nn.Linear(3, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),

            nn.Linear(256, 1),
            #相比于PI_net，这里只有一个输出，用于估计状态值函数。
        )

        self.optim = torch.optim.Adam(self.parameters(), lr=LR_v)
 
    def forward(self, x):
        x = self.net(x)
        #相比于PI_net，这里并计算均值和标准差，而是直接输出状态值函数的估计值。
        
        return x

这个 V_net 类定义了一个神经网络模型，用于在强化学习中作为价值网络。价值网络用于估计给定状态的预期回报。

当 `x` 经过神经网络后变成状态价值的估计，并不是因为神经网络本身有什么特别之处，而是因为神经网络被训练来学习这种映射关系。以下是其中的关键点：

1. **学习映射**：神经网络通过训练数据学习将输入状态 `x` 映射到相应的状态价值估计。训练过程中，神经网络的参数（权重和偏置）被调整，以最小化价值估计和实际回报之间的差异。

2. **优化目标**：在训练价值网络时，通常使用某种形式的损失函数（例如均方误差损失），它衡量价值估计的准确性。优化器（如Adam）使用梯度下降来更新网络参数，以减少这个损失。

3. **泛化能力**：神经网络具有泛化能力，这意味着它们可以对未见过的状态进行价值估计。这是因为网络学习了从状态到价值的一般映射规律，而不仅仅是记忆训练数据。

4. **非线性变换**：神经网络通过非线性激活函数（如ReLU）引入非线性，使得网络能够学习和模拟复杂的映射关系，这些关系可能无法用线性模型来表示。

5. **多层结构**：神经网络通常包含多个隐藏层，这增加了网络的容量，使其能够捕捉更复杂的特征和模式。

6. **价值函数的形式**：状态价值函数 \( V(s) \) 通常是一个标量值，表示从状态 `s` 开始并遵循某个策略所能获得的预期回报。神经网络的最后一层通常是一个线性层，输出一个单一的标量值，代表这个估计。

7. **训练数据**：神经网络的训练数据通常包括状态、对应的回报和折扣因子等信息。这些数据可以来自于与环境的交互，或者是通过其他方法（如时间差分学习）生成。

8. **网络结构**：虽然 `V_net` 中的网络结构（多个全连接层和ReLU激活函数）在深度学习中很常见，但关键在于如何使用这些层来学习状态价值映射。网络结构的选择和设计会影响学习效果和性能。

总之，神经网络通过学习大量的训练数据来掌握从状态到价值的映射，使得它能够对任意输入状态 `x` 给出一个估计的价值。这种能力是通过神经网络的训练和优化过程实现的，而不是网络结构本身固有的。

这个 Agent 类实现了PPO算法中的核心组件，并结合了Actor-Critic方法的一些特点。

初始化 (__init__ 方法)：
创建策略网络 (self.pi) 和价值网络 (self.v)，以及它们对应的旧网络 (self.old_pi 和 self.old_v)，用于在更新过程中计算策略和价值的变化。

选择动作 (choose_action 方法)：
使用旧策略网络 self.old_pi 来选择动作，这对应于PPO算法中使用旧策略采样动作以计算目标函数。

存储经验 (push_data 方法)：
将与环境交互获得的状态、动作、奖励等信息存储起来，用于后续的策略和价值网络的更新。

采样经验 (sample 方法)：
从存储的经验中采样出一个批次的数据，用于训练。这是PPO算法中批量更新策略和价值网络的常见做法。

更新网络 (updata 方法)：
这是PPO算法的核心更新步骤，包括以下几个关键操作：
计算时间差目标 (td_target)：使用奖励和下一个状态的价值估计来计算时间差目标，用于更新价值网络。
计算优势估计 (A)：使用GAE (Generalized Advantage Estimation) 方法计算优势函数，用于更新策略网络。
策略网络更新：计算PPO-Clip目标函数，使用截断的概率比率来更新策略网络。
价值网络更新：使用均方误差来更新价值网络，使其更准确地预测状态价值。

保存和加载模型 (save 和 load 方法)：
用于保存训练好的模型参数到文件，以及从文件中加载预训练的模型参数。

PPO-Clip目标函数：
在 updata 方法中，通过计算 L1 和 L2 来实现PPO-Clip目标函数，这是PPO算法的关键特性，用于限制策略更新的幅度。

优化器步骤：
使用PyTorch的优化器（如Adam）来执行梯度下降，更新策略和价值网络的参数。

更新旧网络：
在每次更新后，将当前网络的参数复制到旧网络，为下一次迭代做准备。
这个 Agent 类实现了PPO算法的主要逻辑，包括策略和价值网络的构建、经验的收集与采样、网络的更新以及模型的保存和加载。通过这种方式，智能体能够在与环境的交互中学习并优化其策略。

In [ ]:
class Agent(object):
    # 定义了一个新的类 Agent，它继承自 object
    #（在Python 3中，所有类都隐式地继承自 object，所以这里的 (object) 是可选的）。

    def __init__(self):
        self.v = V_net()
        #价值网络
                
        self.pi = Pi_net()
        #策略网络

        self.old_pi = Pi_net()        
        #旧策略网络

        self.old_v = V_net()            
        #旧价值网络    用于计算上次更新与下次更新的差别 

        #ratio

        self.load()
        #调用智能体的 load 方法，尝试从文件中加载预训练的模型参数。

        self.data = []               
        #用于存储经验（状态s、动作a、奖励r等）。

        self.step = 0
        #一个计数器，记录智能体的训练步骤。
 
    def choose_action(self, s):
    #定义了智能体选择动作的方法。使用旧策略网络 old_pi 来选择动作，并且不计算梯度。
    #这是因为在训练过程中，我们需要使用旧策略网络来生成动作，以确保梯度计算的正确性。
    #在这个方法中，我们首先将  状态张量 s  传递给 old_pi 网络，得到动作的均值 mu 和标准差 sigma。
    #然后，我们使用 mu 和 sigma 构建一个正态分布 dis，并从中采样出一个动作 a。
    #最后，我们返回动作 a 的值。

        with torch.no_grad():
        #这个上下文管理器指示PyTorch在其中不计算或存储梯度，通常用于推理或选择动作时。
        #这意味着在这个代码块内进行的所有操作都不会追踪历史记录（即不会生成计算图），因此不会计算关于这些操作的梯度。

            mu, sigma = self.old_pi(s)
            #通过旧策略网络 old_pi 计算状态 s 对应的  动作的均值 mu 和标准差 sigma。

            dis = torch.distributions.normal.Normal(mu, sigma)        
            #使用均值和标准差创建一个正态分布。
            #均值和标准差的正态分布，是怎么出现动作的？

            a = dis.sample()   
            #采样出一个动作

        print("State tensor shape:", s.shape)  
        # 添加这行来打印  状态张量  的形状

        return a.item()
        #返回采样动作的标量值。
 
    def push_data(self, transitions):
        #定义了一个 push_data 方法，用于将经验存储到 self.data 中。
        #这个方法接受一个 transitions 参数，
        #它是一个元组，包含了一个经验样本的所有元素（状态、动作、奖励、下一个状态和完成标志）。
        #这个方法将 transitions 添加到 self.data 中。

        self.data.append(transitions)
 
    def sample(self):
    #定义了一个 sample 方法，用于从 self.data 中采样出一个批次的经验。
    #这个方法首先创建了一个空列表，用于存储批次中的所有元素。
    #然后，它遍历 self.data 中的每一个经验样本，
    #将其中的状态、动作、奖励、下一个状态和完成标志分别存储到不同的列表中。
    #接着，它将这些列表转换为张量，并将它们拼接成一个批次。
    #最后，它清空 self.data，并返回这个批次的所有元素。

        l_s, l_a, l_r, l_s_, l_done = [], [], [], [], []
        #（状态、动作、奖励、下一个状态和完成标志）

        for item in self.data:
        #会好奇，为什么会有很多个item

            s, a, r, s_, done = item
            l_s.append(torch.tensor(np.array([s]), dtype=torch.float))
            #首先，将变量 s 转换成一个 NumPy 数组。
            #然后，使用 torch.tensor 将 NumPy 数组转换成 PyTorch 张量。
            #接着，使用 dtype=torch.float 将张量的数据类型设置为 float。
            #最后，新创建的张量被追加到列表 l_s 中。l_s 是一个用于存储状态张量的列表，这些状态张量可能在强化学习中用于收集经验回放数据。

            l_a.append(torch.tensor(np.array([[a]]), dtype=torch.float))
            l_r.append(torch.tensor(np.array([[r]]), dtype=torch.float))
            l_s_.append(torch.tensor(np.array([s_]), dtype=torch.float))
            l_done.append(torch.tensor(np.array([[done]]), dtype=torch.float))

        #将所有列表中的张量按第一个维度（0维）连接起来。
        #还是有点没有理解什么叫做  按照第一个维度连接起来
        s = torch.cat(l_s, dim=0)
        a = torch.cat(l_a, dim=0)
        r = torch.cat(l_r, dim=0)
        s_ = torch.cat(l_s_, dim=0)
        done = torch.cat(l_done, dim=0)

        self.data = []
        return s, a, r, s_, done

 
    def updata(self):
    #定义了一个 updata 方法，用于更新智能体的策略和价值网络。
    #在这个方法中，我们首先从 self.data 中采样出一个批次的经验。
    #然后，我们使用这个批次的经验来更新策略网络和价值网络。
    #具体来说，我们首先计算目标值 td_target，用于更新价值网络。
    #然后，我们计算优势估计 A，用于更新策略网络。
    #接着，我们使用这些目标值来计算策略网络和价值网络的损失。
    #最后，我们使用这些损失来更新策略网络和价值网络的参数。
    #在更新完成后，我们将策略网络和价值网络的参数复制给旧策略网络和旧价值网络。

        self.step += 1
        s, a, r, s_, done = self.sample()

        #接下来的循环是Soft Actor-Critic算法的更新过程，
        #包括  计算时间差目标 td_target，计算策略的损失 loss_pi 和价值的损失 loss_v，然后执行反向传播和优化器步骤。
        for _ in range(K_epoch):
        #在每个周期中，我们执行 K_epoch 次优化迭代。
        
            with torch.no_grad():
                '''loss_v'''
                td_target = r + GAMMA * self.old_v(s_) * (1 - done)
                #计算时间差目标 td_target，用于更新价值网络。
                '''loss_pi'''
                mu, sigma = self.old_pi(s)
                old_dis = torch.distributions.normal.Normal(mu, sigma)
                log_prob_old = old_dis.log_prob(a)
                td_error = r + GAMMA * self.v(s_) * (1 - done) - self.v(s)
                td_error = td_error.detach().numpy()
                A = []
                adv = 0.0
                for td in td_error[::-1]:
                    adv = adv * GAMMA * LAMBDA + td[0]
                    A.append(adv)
                A.reverse()
                A = torch.tensor(A, dtype=torch.float).reshape(-1, 1)
 
            mu, sigma = self.pi(s)
            new_dis = torch.distributions.normal.Normal(mu, sigma)
            log_prob_new = new_dis.log_prob(a)
            ratio = torch.exp(log_prob_new - log_prob_old)
            L1 = ratio * A
            L2 = torch.clamp(ratio, 1 - CLIP, 1 + CLIP) * A
            loss_pi = -torch.min(L1, L2).mean()
            self.pi.optim.zero_grad()
            loss_pi.backward()
            self.pi.optim.step()
 
            loss_v = F.mse_loss(td_target.detach(), self.v(s))
 
            self.v.optim.zero_grad()
            loss_v.backward()
            self.v.optim.step()

        self.old_pi.load_state_dict(self.pi.state_dict())
        self.old_v.load_state_dict(self.v.state_dict())
        #更新旧策略网络和旧价值网络的参数。
 
    def save(self):
        #定义了一个 save 方法，用于保存智能体的策略和价值网络参数到文件中。
        #这个方法调用了 PyTorch 的 torch.save 函数，将策略网络和价值网络的参数保存到 pi.pth 和 v.pth 文件中。

        torch.save(self.pi.state_dict(), 'pi.pth')
        torch.save(self.v.state_dict(), 'v.pth')
        print('...save model...')
 
    def load(self):
        #定义了一个 load 方法，用于从文件中加载预训练的策略和价值网络参数。
        #这个方法尝试从 pi.pth 和 v.pth 文件中加载参数，如果加载成功则打印“...load...”。
        try:
            self.pi.load_state_dict(torch.load('pi.pth'))
            self.v.load_state_dict(torch.load('v.pth'))
            print('...load...')
        except:
            pass

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def main():
    agent = Agent()
    agent.load()
    max_rewards = -1000000
    rewards_list = [] # 用于存储每个周期的奖励值
    for _ in range(EP_MAX):
        
        s = env.reset()  # 确保这里的s不是空的
        
        start = True
        rewards = 0
        while start:
            for i in range(HORIZON):
                #env.render()
                # 确保s是一个非空的序列，并且正确地创建了一个张量
                s_tensor = torch.tensor(s, dtype=torch.float)
                a = agent.choose_action(s_tensor)
                print(env.step([a]))
                s_, r, done, info = env.step([a])
                rewards += r

                agent.push_data((s, a, (r + 8.1) / 8.1, s_, done))
                
                if done:
                    start = False
                    break
                s = s_
            agent.updata()
            rewards_list.append(rewards) 
        if _ % 10 == 0:
            print(_, ' ', rewards, ' ', agent.step)
        if max_rewards < rewards:
            max_rewards = rewards
            agent.save()

    # 绘制奖励变化图
    plt.figure(figsize=(10, 5))
    plt.plot(rewards_list, label='Rewards over Episodes')
    plt.xlabel('Episode')
    plt.ylabel('Cumulative Reward')
    plt.title('Rewards over Episodes')
    plt.legend()
    plt.show()

if __name__ == '__main__':
    main()